## Azure hosted OpenAI and LangChain

your .env file will need these

```
AZURE_OPENAI_API_KEY="YOUR KEY"
AZURE_OPENAI_API_INSTANCE_NAME="https://YOUR_AZURE_KEY_ENDPOINT.com"
AZURE_OPENAI_API_DEPLOYMENT_NAME="THE_DEPLOYMENT_NAME"
AZURE_OPENAI_API_TYPE="azure"
AZURE_OPENAI_API_VERSION="2023-05-15"
```

In [1]:
! pip install -q openai==0.27.8 langchain==0.0.227 python_dotenv
import os
import openai
import langchain
from dotenv import load_dotenv

In [4]:
load_dotenv(".env", override=True)

openai.api_key = os.environ['AZURE_OPENAI_API_KEY']
openai.api_base = os.environ['AZURE_OPENAI_API_INSTANCE_NAME']
openai.default_model = os.environ['AZURE_OPENAI_API_DEPLOYMENT_NAME']
openai.api_version = os.environ['AZURE_OPENAI_API_VERSION']
openai.api_type = os.environ['AZURE_OPENAI_API_TYPE']


# from langchain.chat_models import ChatOpenAI
# llm = ChatOpenAI(
#     temperature=0.3,
#     model=default_model
# )

from langchain.chat_models import AzureChatOpenAI


default_model = openai.default_model

llm = AzureChatOpenAI(
    deployment_name=default_model,
    model_name=default_model,
    openai_api_base=openai.api_base,
    openai_api_key=openai.api_key,
    openai_api_type=openai.api_type, 
    openai_api_version=openai.api_version
)



In [5]:
## Let's do a simple completion
llm.predict("The prime minister of the United Kingdom is ",
    engine=default_model)

'Boris Johnson.'

In [6]:
## You could do a more complex completion and it would handle it with no issues
chat= """
Lou Costello: All I’m trying to find out is what’s the guy’s name on first base.

Bud Abbott: No. What is on second base.

Lou Costello: I’m not asking you who’s on second.

Bud Abbott: Who’s on first.

Lou Costello: One base at a time!

Bud Abbott: Well, don’t change the players around.

Lou Costello: I’m not changing nobody!

Bud Abbott: Take it easy, buddy.

Lou Costello: I’m only asking you, who’s the guy on first base?

Bud Abbott: That’s right.

Lou Costello: Ok.

Bud Abbott: All right.

Lou Costello: What’s the guy’s name on first base?

Bud Abbott:
"""

### The large language model is not always funny
llm.predict(chat,
    engine = default_model,
    temperature=0)


'That’s right.'

In [5]:
## Chat bot With memory in LangChain with OpenAI

from langchain.chains import LLMChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferWindowMemory

import json
# pretty printing JSON objects
def json_pretty(input_object):
  print(json.dumps(input_object, indent=4))


import textwrap
# wrap text when printing, because colab scrolls output to the right too much
def wrap_text(text, width):
    wrapped_text = textwrap.wrap(text, width)
    return '\n'.join(wrapped_text)

template = """The following is a serious conversation between a human and a TV
News Anchor named Newsy McNewserson.
The Anchor provides autoritative information and commentary in short responses.
If the Anchor does not know the answer to a question,
it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
Anchor:"""

MEMORY = ConversationBufferWindowMemory(ai_prefix="Anchor", k=2)

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = LLMChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=MEMORY
)

def chatLoop():
  print(" -- Have a conversation with a TV news Anchor: ")
  print(" -- Ask this AI \"what is in the news?\" ")
  print(" -- type 'exit' when done")

  user_input = input("> ")
  while not user_input.lower().startswith("exit"):
      print( conversation.run(input=user_input) )
      print(" -- type 'exit' when done")
      user_input = input("> ")
  print("\n -- end conversation --")
     

In [6]:
## start a new chat each time
MEMORY.clear()
## start the chat
chatLoop()
     

 -- Have a conversation with a TV news Anchor: 
 -- Ask this AI "what is in the news?" 
 -- type 'exit' when done


> Entering new  chain...
Prompt after formatting:
The following is a serious conversation between a human and a TV
News Anchor named Newsy McNewserson.
The Anchor provides autoritative information and commentary in short responses.
If the Anchor does not know the answer to a question,
it truthfully says it does not know.

Current conversation:

Human: hello
Anchor:

> Finished chain.
Hello there. How can I assist you today?
 -- type 'exit' when done

 -- end conversation --
